# AWS S3 Operations - Comprehensive Guide

This notebook provides a complete walkthrough of AWS S3 operations using boto3 SDK.

## What You'll Learn

1. **S3 Fundamentals** - Understanding buckets and objects
2. **Bucket Operations** - List, create, and manage buckets
3. **Object Operations** - Upload, download, list, and delete files
4. **Advanced Features** - Metadata, presigned URLs, progress tracking
5. **Error Handling** - Robust exception management
6. **Best Practices** - Security and performance optimization

## S3 Architecture Overview

```
┌─────────────────────────────────────────────────────────┐
│  AWS S3 STRUCTURE                                       │
│                                                         │
│  Your AWS Account                                       │
│  └─► Buckets (Globally unique names)                    │
│      ├─► Bucket 1: my-app-data-2024                     │
│      │   ├─► Object: file1.txt                          │
│      │   ├─► Object: images/photo.jpg                   │
│      │   └─► Object: data/2024/records.csv              │
│      │                                                  │
│      └─► Bucket 2: backups-prod                         │
│          ├─► Object: backup-2024-01.tar.gz              │
│          └─► Object: logs/app.log                       │
│                                                         │
│  Key Concepts:                                          │
│  • Bucket = Container (like a root folder)              │
│  • Object = File with metadata                          │
│  • Key = Full path/name (e.g., "folder/file.txt")       │
│  • Region = Geographic location                         │
│  • Storage Class = Cost/access tier                     │
└─────────────────────────────────────────────────────────┘
```

In [6]:
import boto3
from dotenv import load_dotenv
load_dotenv()
import os

## 1. Environment Setup

Loading AWS credentials securely from environment variables using `.env` file.

**Security Best Practice:** Never hardcode credentials in your code!

In [9]:
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")    
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")

## 2. AWS Credentials Configuration

Required environment variables in your `.env` file:
```
AWS_BUCKET_NAME=your-bucket-name
AWS_ACCESS_KEY=your-access-key-id
AWS_SECRET_KEY=your-secret-access-key
AWS_REGION=us-east-1
```

## 3. Initialize S3 Client

boto3 provides two interfaces:
- **Client**: Low-level service access (more control)
- **Resource**: Higher-level object-oriented interface (easier to use)

We'll use the **client** interface for maximum flexibility.

In [10]:
# create an s3 client
s3_client = boto3.client('s3', region_name=AWS_REGION, 
                         aws_access_key_id=ACCESS_KEY, 
                         aws_secret_access_key=SECRET_KEY)

## 4. Bucket Operations

### Operation Flow
```
┌─────────────────────────────────────────┐
│  BUCKET LIFECYCLE                       │
│                                         │
│  1. CREATE → Bucket exists in S3        │
│  2. LIST → View all your buckets        │
│  3. CONFIGURE → Set permissions/policies│
│  4. USE → Upload/download objects       │
│  5. DELETE → Remove bucket (if empty)   │
└─────────────────────────────────────────┘
```

## 5. Create Bucket

Create a new S3 bucket in your account.

**Important Rules:**
- Bucket names must be globally unique across ALL AWS accounts
- Only lowercase letters, numbers, hyphens, and dots allowed
- Must be 3-63 characters long
- Cannot start or end with a hyphen
- Region matters for latency and compliance

**Regions:**
- `us-east-1` is the default and doesn't require LocationConstraint
- Other regions need explicit LocationConstraint configuration

In [11]:
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region=None):
    """
    Create an S3 bucket in a specified region
    
    Args:
        bucket_name (str): Name for the bucket (must be globally unique)
        region (str): AWS region (if None, uses default from client)
    
    Returns:
        bool: True if bucket created, False otherwise
    """
    try:
        if region is None or region == 'us-east-1':
            # us-east-1 doesn't require LocationConstraint
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            # Other regions require LocationConstraint
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"SUCCESS: Bucket '{bucket_name}' created successfully")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'BucketAlreadyExists':
            print(f"ERROR: Bucket '{bucket_name}' already exists (owned by someone else)")
        elif error_code == 'BucketAlreadyOwnedByYou':
            print(f"INFO: Bucket '{bucket_name}' already exists and is owned by you")
        else:
            print(f"ERROR: Failed to create bucket - {e}")
        return False

# Example usage (uncomment to test):
create_bucket(f"{BUCKET_NAME}-test", region=AWS_REGION)

SUCCESS: Bucket 'real-learn-s3-test' created successfully


True